In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
# Wikipedia as one source
api_wrapper = WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki_tool.name

'wikipedia'

In [3]:
# This website used as other source : https://docs.langchain.com/
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings

URL = "https://docs.langchain.com/"
loader = WebBaseLoader(URL)
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectordb = FAISS.from_documents(documents, embeddings)
retriever = vectordb.as_retriever()
retriever

C:\Users\hp\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001EEFBA6AA20>, search_kwargs={})

In [4]:
from langchain_core.tools import Tool

retriever_tool = Tool(
    name="langsmith_search",
    func=lambda query: retriever.invoke(query),
    description="Use only this tool when you need to answer questions about LangSmith",
)

retriever_tool.name

'langsmith_search'

In [5]:
# %pip install arxiv
# Arxiv as another source
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_api_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=500)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)
arxiv_tool.name

'arxiv'

In [6]:
tools = [retriever_tool, wiki_tool, arxiv_tool]
tools

[Tool(name='langsmith_search', description='Use only this tool when you need to answer questions about LangSmith', func=<function <lambda> at 0x000001EEFBA79EE0>),
 WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'C:\\Users\\hp\\AppData\\Roaming\\Python\\Python312\\site-packages\\wikipedia\\__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=500))]

In [7]:
# Create the LLM
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="llama3")
llm

OllamaLLM(model='llama3')

In [ ]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="llama3")
llm

C:\Users\hp\AppData\Local\Temp\ipykernel_1468\4086057440.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3")  # or whatever model you're using


ChatOllama(model='llama3')

In [8]:
# Creating the prompt
prompt = ""

In [ ]:
# Creating the agent with tools
from langchain.agents import create_agent
agent = create_agent(llm, tools=tools)
agent.invoke(
    {
        "input": "Give me a brief overview of LangSmith."
    }
)

NotImplementedError: 